In [5]:
import numpy as np
from matplotlib import pyplot as plt 
import math
from scipy import integrate

LENSFOCAL = 502.5 # this is in mm
ZERO_X = 5280/2
ZERO_Y = 4600 /2

In [6]:
bessy= np.loadtxt("bessy.slp")
bessy_slope = bessy [:,1]

#print(bessy_slope)
#arrayX = bessy[:,0]

In [7]:
ltp_data = np.loadtxt("FullData14-17_20250121.txt", skiprows = 7)
slope_file = np.loadtxt("Xpos_slopes14-17_20250121.txt", skiprows = 1 )
slope_data = slope_file[:,1]
slope_X = slope_file[:,0]
slopeXpos = slope_X - slope_X[0]
#print(slope_data)
#ltp_data = np.loadtxt("FullData16-58_20250116.txt", skiprows = 7) this is the best dataset so far. 
#LTP data is X pos, Y Pos, Average Centroid X, Average Centroid Y, so I need the last column)
avgX = ltp_data[:,2]
avgY = ltp_data[:,3]
#print(ltp_data)
x_column = ltp_data[:,0]
diff = x_column[-1] - x_column[0]
print(diff)
startPos = x_column[0]
Xpos = np.arange(0,len(x_column), 1)

#print(avgX - avgY)


FileNotFoundError: FullData14-17_20250121.txt not found.

In [10]:
print(len(x_column))
alan = np.arange(0,len(x_column), 1)
print(alan)

241
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240]


In [9]:
def slope_calc(value, focal):
    slope_error = 0.5 * (math.atan((2.74 * (ZERO_Y - value))/(focal* 1000))) # this is in microns, so in µrad? 
    return(slope_error)
def my_fit (arrayX, arrayY, order):
    coeff = np.polyfit(arrayX, arrayY, order)
    p = np.poly1d(coeff)
    fit = p(arrayX)
    radius = 1 /coeff[0]
    return(fit, radius)
def RMS(array):
    RMS = np.sqrt(np.mean(array**2))
    return(RMS)

def figure_error(arrayX, arrayY):
    """
    Calculates the figure error given and array of positions/slopes
    :param arrayX: y-position of the centroid? or X-position of the head?
    :param arrayY: slopes
    :return: an array, heights.
    """

    heights = []
    heights = integrate.cumtrapz(arrayX, arrayY, initial=0)
    return (heights)

In [10]:
slope_array = np.zeros(len(avgY))
for i in range (len(avgY)):
    slope_array [i] = slope_calc(avgY[i], focal = 400)
    #print(slope_array[i])

"""oldX = arrayX
arrayX = oldX[:-1].copy()
print(len(slope_array))
print(len(arrayX))
print(len(avgY))"""
fit, radius = my_fit(Xpos, slope_array, 1)

NameError: name 'avgY' is not defined

In [8]:
plt.figure()
#plt.plot(Xpos/1000, fit * 1000, label = "myfit")
#plt.plot(Xpos/1000, (slope_array - fit) *1000000, label = "myslope - myfit, f = 400")
#plt.plot(Xpos/1000, slope_array*1000, label = "slope array, unfitted", marker = "d")
#plt.plot(Xpos, slope_array * 1000, label = "myslope")
plt.plot(Xpos/1000, bessy_slope, label = "Bessy")

plt.xlabel("X position [mm]")
plt.ylabel("Slope erorr [µrad]")
plt.legend()
plt.show()

NameError: name 'Xpos' is not defined

<Figure size 640x480 with 0 Axes>

In [4]:
slope_array_600 = np.zeros(len(avgY))

for i in range (len(avgY)):
    slope_array_600 [i] = slope_calc(avgY[i], focal = 600.0)
    #print(slope_array[i])
fit_600, radius_600 = my_fit(Xpos, slope_array_600, 1)

slope_array_500 = np.zeros(len(avgY))
for i in range (len(avgY)):
    slope_array_500 [i] = slope_calc(avgY[i], focal = 502.5)
    #print(slope_array[i])
fit_500, radius_500 = my_fit(Xpos, slope_array_500, 1)

slope_array_400 = np.zeros(len(avgY))

for i in range (len(avgY)):
    slope_array_400 [i] = slope_calc(avgY[i], focal = 400.0)
    #print(slope_array[i])
fit_400, radius_400 = my_fit(Xpos, slope_array_400, 1)

slope, fit = my_fit(slopeXpos, slope_data, 1)
heights = figure_error(Xpos,slope_array_500 - fit_500 )
rms_heights = RMS(heights)

    
"""oldX = arrayX
arrayX = oldX[:-1].copy()
print(len(slope_array))
print(len(arrayX))
print(len(avgY))
fit, radius = my_fit(Xpos, slope_array, 1)"""
plt.figure()
plt.plot(Xpos/1000, (slope_array_600 - fit_600) *1000000, label = "RMS slope = 1.88 µrad")
#plt.plot(Xpos/1000, (slope - fit )* 1000000, label = "Slope from LTP, f = 502.5 mm")
#plt.plot(Xpos, slope_array * 1000, label = "myslope")
plt.plot(Xpos/1000, bessy_slope, label = "Bessy slope = 0.8688 µrad")

plt.xlabel("X position [mm]")
plt.ylabel("Slope erorr [µrad]")
plt.legend()
plt.show()
heights_bessy = figure_error(Xpos, bessy_slope)
print(RMS(slope_array_500-fit_500))
print(rms_heights)
print("bessy heights rms: ", RMS(heights_bessy)/1000)
print(RMS(bessy_slope))

NameError: name 'avgY' is not defined